# Group 3

Dave Carlson, Brooke, Daniel

## Imports and set up LLM client

In [ ]:
import sys
import os
import json

from IPython.display import display, Markdown, Code

try:
    project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
except IndexError:
    project_root = os.path.abspath(os.path.join(os.getcwd()))

if project_root not in sys.path:
    sys.path.append(project_root)

from utils import setup_llm_client, prompt_enhancer, get_completion

llm_model = "gpt-4o"

client, model_name, api_provider = setup_llm_client(llm_model)

def display_markdown(text):
    display(Markdown(text))

def display_code(code, language='json'):
    display(Code(code, language=language))

write_artifacts = False

## Problem Statement

In [ ]:
problem_statement = """
I am seeking a web application that can help me manage my personal finances more effectively. 
- The application should allow me to track my income and expenses, create budgets, and generate reports to visualize my spending patterns. 
- Additionally, I would like the app to provide reminders for upcoming bills and suggest ways to save money based on my spending habits. 
- The interface should be user-friendly and accessible from both desktop and mobile devices.
"""

### Enhanced Problem Statement

In [ ]:
enhanced_problem_statement = prompt_enhancer(problem_statement)
display_markdown(enhanced_problem_statement)

## Brainstorm Features

In [ ]:
# Set the output directory based on API provider and model
output_dir = f"../artifacts/documents/{api_provider}/{llm_model}"
os.makedirs(output_dir, exist_ok=True)

# Brainstorm features prompt
brainstorm_prompt = f"""
Based on the problem statement, brainstorm a list of potential features that could address the problem.
- Keep the list concise and focused on high-impact features.
- Provide a brief description for each feature.
- Format the output in a markdown list format without the ```markdown``` fences.

<Problem Statement>
{enhanced_problem_statement}
</Problem Statement>
"""

brainstormed_features = get_completion(brainstorm_prompt, client, model_name, api_provider)
display(Markdown(brainstormed_features))

if write_artifacts:
    with open(os.path.join(output_dir, "brainstormed_features.md"), "w") as f:
        f.write(brainstormed_features)

## Generate User Personas

In [ ]:
personas_prompt = f"""
<Prompt Instructions>
Based on the problem statement and the brainstormed features, identify at least three, but no more than five, unique user personas who would benefit from this tool. 
- The roles and their experience should vary between each persona. 
- Include a mix of technical and non-technical users.
- Provide the following for each persona:
  - Name
  - Role
  - Experience Level
  - Goals
  - Challenges
  - How this tool can help
- Format the output in a markdown list format without the ```markdown``` fences.
</Prompt Instructions>

<Problem Statement>   
{enhanced_problem_statement}
</Problem Statement>

<Brainstormed Features>
{brainstormed_features}
</Brainstormed Features>"""

user_personas = get_completion(personas_prompt, client, model_name, api_provider)
display_markdown(user_personas)

if write_artifacts:
    with open(os.path.join(output_dir, "user_personas.md"), "w") as f:
        f.write(user_personas)

## Generate User Stories

In [ ]:
personas_prompt = f"""
<Prompt Instructions>
You are a senior product manager.
- Based on the problem statement, brainstormed features, and user personas, create at least three but no more than five user stories per persona.
- Your output should be a properly-structured JSON with no markdown fences.
- Each user story should contain the following keys:
    - "ID": A unique identifier for the user story in a two-digit format with a leading zero.
    - "persona": The persona this user story is for.
    - "user_story": A brief description of the user story.
    - "acceptance_criteria": A list of at least three acceptance criteria for the user story, written in Gherkin format i.e., given/when/then statements.
- Ensure the output is only valid JSON code and can be parsed without errors.
</Prompt Instructions>

<Problem Statement>
{enhanced_problem_statement}
</Problem Statement>

<Brainstormed Features>
{brainstormed_features}
</Brainstormed Features>

<User Personas>
{user_personas}
</User Personas>"""

print("Generating user stories...")
user_stories = get_completion(personas_prompt, client, model_name, api_provider)

try:
    if "```" in user_stories:
        user_stories = user_stories.split("```")[1].lstrip("json").strip()
        user_stories = json.loads(user_stories)
        print("Successfully parsed user stories JSON.")

    user_stories = json.loads(user_stories)
    print(json.dumps(user_stories, indent=4))
    if write_artifacts:
        with open(os.path.join(output_dir, "user_stories.json"), "w") as f:
            json.dump(user_stories, f, indent=4)

except (json.JSONDecodeError, TypeError, IndexError) as e:
    print(f"Error: Failed to parse LLM output as JSON. \nError: {e}")
    print(f"LLM Output:\n{user_stories}")
    user_stories = []
